In [11]:
!pip install -U selenium
!pip install -U pandas
!pip install -U bs4

Requirement already up-to-date: selenium in c:\users\alexi\anaconda3\lib\site-packages (3.141.0)
Requirement already up-to-date: pandas in c:\users\alexi\anaconda3\lib\site-packages (1.0.3)
Requirement already up-to-date: bs4 in c:\users\alexi\anaconda3\lib\site-packages (0.0.1)


In [12]:
# Importing Packages:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time

import pandas as pd
from bs4 import BeautifulSoup

In [13]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')

driver = webdriver.Chrome(options = options)

In [17]:
driver.get('https://www.renodepot.com/webapp/wcs/stores/servlet/CategorySearchDisplay?pageSize=infinite&catalogId=10551&top=Y&urlLangId=-1&categoryId=32504&langId=-1&storeId=10701&productCategory=14745&page=1')
driver.refresh()

In [18]:
selectMTL = driver.find_element_by_id("department")
selectMTL.click()

In [19]:
SCROLL_PAUSE_TIME = 2

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [20]:
page_source = driver.page_source

In [21]:
soup = BeautifulSoup(page_source, 'lxml')

In [29]:
products = soup.find_all('a', class_ = 'product_box')
make = []
makeTool = []
deltaPrice = []
normPrice = []
salePrice = []
productURL = []

for product in products:
    make.append(product.find('div', class_ = 'product_manufacturer').get_text().strip())
    makeTool.append(product.find('div', class_ = 'product_name').get_text().strip())
    salePrice.append(product.find('span', class_ = 'integer').get_text().strip())
    productURL.append(product.get('href'))  
    try:
        deltaPrice.append(product.find('div', class_ = 'rebate').get_text().strip())
        normPrice.append(product.find('span', class_ = 'linethrough').get_text().strip())
    except:
        deltaPrice.append('0')
        normPrice.append(product.find('span', class_ = 'integer').get_text().strip())
        continue
        
     

In [30]:
product_df = pd.DataFrame(columns=['Make', 'Tool_Type', 'Regular_Price', 'Sale_Price', 'Rebate', 'Product_URL'])

product_df['Make'] = make
product_df['Tool_Type'] = makeTool
product_df['Regular_Price'] = normPrice
product_df['Sale_Price'] = salePrice
product_df['Rebate'] = deltaPrice
product_df['Product_URL'] = productURL

In [32]:
product_df.sample(10)

,Make,Tool_Type,Regular_Price,Sale_Price,Rebate,Product_URL
673,BOSCH,Bosch Colt™ Variable Speed Electronic Palm Router,371,371,0,https://www.renodepot.com/en/bosch-colt-variab...
10,BOSCH,"Drill/Driver Kit - 1/2"" - 2 18V SlimPack Batte...",$149.00,119,Save $29.80,https://www.renodepot.com/en/fix-me-19836264
636,BOSCH,Bosch 14-Gallon Dust Extractor with HEPA Filter,"1,034","1,034",0,https://www.renodepot.com/en/bosch-14-gallon-d...
647,BOSCH,Bosch Connected Green-Beam Cross-Line Laser - ...,574,574,0,https://www.renodepot.com/en/bosch-connected-g...
511,FORESTO,Framing Nails - Screw Shank - 2 3/8'' - 1000/Box,18,18,0,https://www.renodepot.com/en/framing-nails-scr...
693,BOSCH,Bosch 2-Tool Lithium-Ion Cordless Combo Kit - ...,257,257,0,https://www.renodepot.com/en/bosch-2-tool-lith...
874,BOSCH,Bosch Dual-Bevel Glide Miter Saw - 10'',699,699,0,https://www.renodepot.com/en/fix-me-19835716
586,DEWALT,Laser Pointer - 3-Beam,255,255,0,https://www.renodepot.com/en/3-beam-laser-poin...
204,CRAFTSMAN,Cordless Hammer Drill - 20 V MAX,$179.00,143,Save $35.80,https://www.renodepot.com/en/cordless-hammer-d...
441,IRWIN,Standard Marking Chalk,6,6,0,https://www.renodepot.com/en/standard-marking-...


In [36]:
import re

In [43]:
product_df['Regular_Price'] = product_df['Regular_Price'].str.replace(r'\$', '')
product_df['Rebate'] = product_df['Rebate'].str.replace(r'Save \$', '')

In [44]:
product_df.head()

,Make,Tool_Type,Regular_Price,Sale_Price,Rebate,Product_URL
0,METABO HPT,"Pneumatic Stripe Brad Nailer - 5/8""-2""",179.00,89,90.00,https://www.renodepot.com/en/hitachi-pneumatic...
1,MAKITA,Set of 2 Cordless Tools 18V,319.00,189,130.00,https://www.renodepot.com/en/set-of-2-cordless...
2,MAKITA,Set of 2 Cordless Tools - 18 V Lithium-ion,399.00,249,150.00,https://www.renodepot.com/en/set-of-2-cordless...
3,BOSCH,"Small Angle Grinder - 4 1/2"" - Set of 2",149.00,99,50.00,https://www.renodepot.com/en/small-angle-grind...
4,BOSCH,Laser Measuring Tool - 100 ft,89.99,59,30.00,https://www.renodepot.com/en/laser-measuring-t...
